In [1]:
import os
import cv2
import numpy as np
import joblib
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [3]:
# Function to extract color histogram features from an image
def extract_features(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (128, 128))  # Resize for consistency
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

In [5]:
# Function to load dataset
def load_dataset(directory):
    data = []
    labels = []
    class_names = os.listdir(directory)
    for class_id, class_name in enumerate(class_names):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            for file in os.listdir(class_path):
                file_path = os.path.join(class_path, file)
                features = extract_features(file_path)
                data.append(features)
                labels.append(class_id)
    return np.array(data), np.array(labels), class_names

In [7]:
# Load and preprocess data (training, validation, and test sets)
train_dir = "/Users/nasrinjahan/Desktop/Indivisual Project/Project_NK/machineLearning_PlantDiseaseDetectionSystem/Plant_Diseases_Dataset/train"
valid_dir = "/Users/nasrinjahan/Desktop/Indivisual Project/Project_NK/machineLearning_PlantDiseaseDetectionSystem/Plant_Diseases_Dataset/valid"
test_dir = "/Users/nasrinjahan/Desktop/Indivisual Project/Project_NK/machineLearning_PlantDiseaseDetectionSystem/Plant_Diseases_Dataset/test"

In [9]:
X_train, y_train, class_names = load_dataset(train_dir)
X_test, y_test, _ = load_dataset(test_dir)
X_valid, y_valid, _ = load_dataset(valid_dir)

In [10]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

In [13]:
# remove DS.store from the dataset before training the modelas per the last model result
import os

for folder in [train_dir, test_dir, valid_dir]:
    ds_store_path = os.path.join(folder, ".DS_Store")
    if os.path.exists(ds_store_path):
        os.remove(ds_store_path)

In [17]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

In [19]:
# Train SVM model
svm_model = SVC(kernel='linear', C=1.0, probability=True)
svm_model.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [21]:
# Evaluate model
y_pred_test = svm_model.predict(X_test)
y_pred_valid = svm_model.predict(X_valid)

In [23]:
print("Test Accuracy:", accuracy_score(y_test, y_pred_test))
print("Validation Accuracy:", accuracy_score(y_valid, y_pred_valid))
print("\nClassification Report:\n", classification_report(y_valid, y_pred_valid, target_names=class_names))

Test Accuracy: 0.0
Validation Accuracy: 0.1566697017983155

Classification Report:
                                                     precision    recall  f1-score   support

                              Strawberry___healthy       0.97      0.99      0.98       456
                                 Grape___Black_rot       0.94      0.96      0.95       472
                             Potato___Early_blight       0.95      1.00      0.97       485
                               Blueberry___healthy       0.95      0.97      0.96       454
                            Corn_(maize)___healthy       1.00      1.00      1.00       465
                              Tomato___Target_Spot       0.93      0.91      0.92       457
                                         .DS_Store       0.00      0.00      0.00       432
                                   Peach___healthy       0.00      0.00      0.00       485
                              Potato___Late_blight       0.02      0.02      0.02      

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/s